# ¿Cuál es la mejor tarifa?

Trabajas como analista para el operador de telecomunicaciones Megaline. La empresa ofrece a sus clientes dos tarifas de prepago, Surf y Ultimate. El departamento comercial quiere saber cuál de las tarifas genera más ingresos para poder ajustar el presupuesto de publicidad.

Vas a realizar un análisis preliminar de las tarifas basado en una selección de clientes relativamente pequeña. Tendrás los datos de 500 clientes de Megaline: quiénes son los clientes, de dónde son, qué tarifa usan, así como la cantidad de llamadas que hicieron y los mensajes de texto que enviaron en 2018. Tu trabajo es analizar el comportamiento de los clientes y determinar qué tarifa de prepago genera más ingresos.

[Te proporcionamos algunos comentarios para orientarte mientras completas este proyecto. Pero debes asegurarte de eliminar todos los comentarios entre corchetes antes de entregar tu proyecto.]

[Antes de sumergirte en el análisis de datos, explica por tu propia cuenta el propósito del proyecto y las acciones que planeas realizar.]

[Ten en cuenta que estudiar, modificar y analizar datos es un proceso iterativo. Es normal volver a los pasos anteriores y corregirlos/ampliarlos para permitir nuevos pasos.]

Primero planeo leer los dataframes, revisar si tiene valores duplicados, ausentes, en su caso eliminarlos, ver cuantas filas, que tipo de datos con el método info, imprimir 5 filas aleatorias con sample(). Con ello, estoy limpiando los datos.

## Inicialización

In [2]:
import pandas as pd
from matplotlib import pyplot as pyplot
from scipy import stats as st
from scipy.stats import ttest_ind
from scipy.stats import levene
import numpy as np
import seaborn as sns

## Cargar datos

In [13]:
calls = pd.read_csv(r'E:\análisis de datos\datasets\megaline_calls.csv')
internet = pd.read_csv(r'E:\análisis de datos\datasets\megaline_internet.csv')
messages = pd.read_csv(r'E:\análisis de datos\datasets\messages.csv')
plans = pd.read_csv(r'E:\análisis de datos\datasets\megaline_tariffs.csv')
users = pd.read_csv(r'E:\análisis de datos\datasets\megaline_users1.csv')

In [14]:
users.head()

,user_id,first_name,last_name,age,city,reg_date,tariff,churn_date
0,1000,Anamaria,Bauer,45,"Atlanta-Sandy Springs-Roswell, GA MSA",2018-12-24,ultimate,NaN
1,1001,Mickey,Wilkerson,28,"Seattle-Tacoma-Bellevue, WA MSA",2018-08-13,surf,NaN
2,1002,Carlee,Hoffman,36,"Las Vegas-Henderson-Paradise, NV MSA",2018-10-21,surf,NaN
3,1003,Reynaldo,Jenkins,52,"Tulsa, OK MSA",2018-01-28,surf,NaN
4,1004,Leonila,Thompson,40,"Seattle-Tacoma-Bellevue, WA MSA",2018-05-23,surf,NaN
